In [31]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from scipy.stats import norm
import seaborn as sns
import os 
import random

In [67]:
datafile = os.path.join(os.path.dirname(os.getcwd()),'Data')
dataNotLabelled = pd.read_csv(datafile+'/large.csv')
jeopardy = pd.read_csv(datafile+'/jeopardy.csv')
data = pd.merge(dataNotLabelled,jeopardy, how='left',on = 'cif')
Individuals = data[data.category == 0].drop(columns = 'category')

In [68]:
X_suspicious = Individuals[Individuals.suspicious == 0].sample(18000)
X_suspicious.head()

,cif,turnover,transaction_count,io_ratio,age,nationality,is_pep,inactive_days_average,inactive_days_max,n_of_accounts,distinct_counterparties,channel_risk,atm_withdrawal,atm_deposit,suspicious
480753,90480753,4544704.27,1,1.000000,49,32,0,118.01,119,2,1,0.361756,7100.29,0.000000,0
381656,90381656,811619.88,2,0.000000,44,32,0,33.58,68,2,2,0.137832,0.00,9019.326060,0
690529,90690529,222515.08,64,0.218750,20,32,0,2.28,80,3,16,0.269352,839.60,838.135908,0
73203,90073203,1805.97,79,0.367089,17,123,0,2.47,107,1,24,0.278001,305.64,1.440994,0
489932,90489932,1341803.12,2,0.000000,26,32,0,68.32,137,2,1,0.325128,0.00,4629.500644,0


In [71]:
X_non_suspicious = Individuals[Individuals.suspicious == 1]
X_non_suspicious.head()

,cif,turnover,transaction_count,io_ratio,age,nationality,is_pep,inactive_days_average,inactive_days_max,n_of_accounts,distinct_counterparties,channel_risk,atm_withdrawal,atm_deposit,suspicious
51,90000051,532462.34,1,1.000000,38,90,0,11.99,24,1,1,0.081074,6439.40,0.000000,1
89,90000089,1420790.10,6,0.500000,41,186,0,27.89,140,4,2,0.008668,58637.41,941.816740,1
129,90000129,711779.47,3,0.666667,37,186,0,39.82,80,3,1,0.130662,53927.19,2771.451155,1
268,90000268,1187499.07,177,0.418079,41,125,0,1.11,99,1,10,0.064417,504.65,8406.113921,1
273,90000273,994734.36,2,0.000000,31,82,0,79.97,80,2,1,0.051460,994734.36,1204.695732,1


In [72]:
X_start = pd.concat(objs = [X_suspicious,X_non_suspicious],axis = 0)
X_start.head()

,cif,turnover,transaction_count,io_ratio,age,nationality,is_pep,inactive_days_average,inactive_days_max,n_of_accounts,distinct_counterparties,channel_risk,atm_withdrawal,atm_deposit,suspicious
480753,90480753,4544704.27,1,1.000000,49,32,0,118.01,119,2,1,0.361756,7100.29,0.000000,0
381656,90381656,811619.88,2,0.000000,44,32,0,33.58,68,2,2,0.137832,0.00,9019.326060,0
690529,90690529,222515.08,64,0.218750,20,32,0,2.28,80,3,16,0.269352,839.60,838.135908,0
73203,90073203,1805.97,79,0.367089,17,123,0,2.47,107,1,24,0.278001,305.64,1.440994,0
489932,90489932,1341803.12,2,0.000000,26,32,0,68.32,137,2,1,0.325128,0.00,4629.500644,0


In [73]:
rng = np.random.RandomState(42)

# Generate train and test splits
X = X_start.iloc[:, 1: -1].values
y = X_start.iloc[:, -1: ].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# fit the model
clf = IsolationForest(n_estimators=100, max_samples=100, contamination=0.5)
clf.fit(X_train)
y_pred_train = clf.predict(X_train)
y_pred_test = clf.predict(X_test)

/Users/lamiakasmi/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:1713: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.add.reduce(sorted[indexer] * weights, axis=axis) / sumval


In [75]:
y_pred_train

array([ 1,  1, -1, ..., -1,  1, -1])

In [ ]:

# plot the line, the samples, and the nearest vectors to the plane
xx, yy = np.meshgrid(np.linspace(-5, 5, 50), np.linspace(-5, 5, 50))
Z = clf.decision_function(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

plt.title("IsolationForest")
plt.contourf(xx, yy, Z, cmap=plt.cm.Blues_r)

b1 = plt.scatter(X_train[:, 0], X_train[:, 1], c='white',
                 s=20, edgecolor='k')
b2 = plt.scatter(X_test[:, 0], X_test[:, 1], c='green',
                 s=20, edgecolor='k')
c = plt.scatter(X_outliers[:, 0], X_outliers[:, 1], c='red',
                s=20, edgecolor='k')
plt.axis('tight')
plt.xlim((-5, 5))
plt.ylim((-5, 5))
plt.legend([b1, b2, c],
           ["training observations",
            "new regular observations", "new abnormal observations"],
           loc="upper left")
plt.show()